# Small project 1

Analyze data from an ASDEX Upgrade experiment: is the divertor neutral pressure dependant of gas fueling, heating power, both, neither?

## 1 - Import the needed data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.style.use('darkgrid')
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 10, 5

First let's import the Divertor Pressure file

In [ ]:
pressure = np.loadtxt('pressure.30733')

In [ ]:
plt.plot(pressure[:,0], pressure[:,1])
plt.show()

In [ ]:
#Check the time scale
print pressure[21,0], pressure[22,0]

In [ ]:
def smooth(data, N=20):
    return np.convolve(data, np.ones(N)/float(N))[(N-1):]

In [ ]:
plt.plot(pressure[:,0], pressure[:,1])
plt.plot(pressure[:,0], smooth(pressure[:,1], 50), color='r')
plt.show()

In [ ]:
smthpres = smooth(pressure[:,1], 50)

In [ ]:
fuelling = np.loadtxt('fuel.30733')

In [ ]:
plt.plot(fuelling[:,0], fuelling[:,1])
plt.show()

In [ ]:
pheat = np.loadtxt('heating_power.30733')
plt.plot(pheat[:,0], pheat[:,1])
plt.show()

# 2 - Producing a DataFrame and cleaning up the data

In [ ]:
import pandas as pd
from scipy.interpolate import interp1d

In [ ]:
common_time = np.arange(1.5, 7.5, 0.01)

Interpolate data for the common time ranges.

In [ ]:
f_interp = interp1d(fuelling[:,0], fuelling[:,1])
new_fuel = f_interp(common_time)
plt.plot(common_time, new_fuel)
plt.show()

Repeat for Heating Power and divertor pressure.

In [ ]:
h_interp = interp1d(pheat[:,0], pheat[:,1])
new_power = h_interp(common_time)
plt.plot(common_time, new_power)
plt.show()

In [ ]:
press_interp = interp1d(pressure[:,0], smthpres)
new_press = press_interp(common_time)
plt.plot(common_time, new_press)
plt.show()

Create the dataframe:

In [ ]:
#Declare the name with unitsstrtime = 'Time [s]'
strTime = 'Time [s]'
strFuel = 'Fuelling [$10^{22}$e/s]'
strPow = 'Power [MW]'
strPres = 'Pressure [$10^{23}$e$m^{-2}$]'

In [ ]:
#Pass a dictionary as input
d = {strTime: common_time,
     strFuel: new_fuel*1e-22,
     strPow: new_power*1e-6,
     strPres: new_press*1e-23}

In [ ]:
df = pd.DataFrame(data=d)

In [ ]:
df.head()

Reorganize columnsn to have time as first column. This will become a feature in future python editions, so we will just circumvent this with a short workaround.

In [ ]:
col_order = [strTime, strFuel, strPow, strPres]

In [ ]:
df = df[col_order]

In [ ]:
df.head()

In [ ]:
df.plot(kind='line', x=strTime, figsize=(10, 5), fontsize=14)
plt.show()

In [ ]:
df.plot(kind='line', x=strTime, figsize=(10, 5), fontsize=14)
plt.show()

To finish cleaning up the data, we must clean out all NaNs, Infs and non-numerical values. We'll start by copying the dataframe into a new one.

In [ ]:
ldf = df.copy(deep=True)

In [ ]:
ldf.dropna(axis=0, inplace=True)

In [ ]:
ldf.plot(kind='line', x=strTime, figsize=(10,5))
plt.show()

## 3 - Model and fitting

$$Pressure = k \times Fuel^{A}\times Pheat^{B}$$
becomes:
$$\log{Pressure} = k' + A \log{Fuel} + B \log{Pheat}$$
Convert to logarithm, but first make a copy of the DataFrame.

In [ ]:
ldf.head()

In [ ]:
ldf[strPow] = np.log(ldf[strPow])
ldf[strFuel] = np.log(ldf[strFuel])
ldf[strPres] = np.log(ldf[strPres])

In [ ]:
ldf.head()

Now we'll define a fitting function.

In [ ]:
#Define a fitting function
def fit_func( (lfuel, lheat), k, a, b):
    return k + a*lfuel + b*lheat

In [ ]:
from scipy.optimize import *

In [ ]:
popt, pcov = curve_fit(fit_func, (ldf[strFuel], ldf[strPow]), ldf[strPres])

In [ ]:
#The optimised parameters of the fit
print "popt:", popt
#One standard deviation errors on the parameters.
perr = np.sqrt(np.diag(pcov))
print "perr:", perr
#The covariance matrix of the parameters
print "pcov:", pcov

In [ ]:
plt.scatter(ldf[strPres], fit_func( (ldf[strFuel], ldf[strPow]), *popt))
plt.show()

In [ ]:
plt.scatter(ldf[strtime], ldf[strpres], label='Pressure', color='blue')
plt.scatter(ldf[strtime], fit_func( (ldf[strfuel], ldf[strpow]), *popt), label='Fit', color='red')
plt.legend()
plt.show()